# 기본 설정

구글 드라이브 마운트

패키지 설치

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install cinemagoer
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
import pandas as pd
import numpy as np
from glob import glob
import os
import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import BertTokenizer, BertModel, BertPreTrainedModel
from datasets import load_dataset, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union
import re

In [4]:
cd /gdrive/MyDrive/Lectures/2024/RecSys/content-based

/gdrive/MyDrive/Lectures/2024/RecSys/content-based


In [5]:
#@title 마이너 패키지 로딩
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

<ipython-input-5-c31c0f4e8541>:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


# u.item 로드 및 플롯 수집

../raw/ml-100k
- u.item
    - 0xe9 를 utf-8 디코딩 하지 못하는 것을 처리
        - 'ignore'

In [6]:
raw_df_ = pd.read_csv('../raw/ml-100k/u.item', sep='|', header=None, encoding_errors='ignore', index_col=False)#, names=['movieId', 'title', 'rel_date','v_rel_date', 'url'])
item_info_df = raw_df_[[0,1,2,4]].copy()
item_info_df.columns=['movieId', 'title', 'release_date', 'imdburl']
item_info_df.set_index('movieId', inplace=True)

In [7]:
from imdb import Cinemagoer

# create an instance of the Cinemagoer class
ia = Cinemagoer()

# # get a movie
# movie = ia.get_movie('0133093')

# # print the names of the directors of the movie
# print('Directors:')
# for director in movie['directors']:
#     print(director['name'])

# # print the genres of the movie
# print('Genres:')
# for genre in movie['genres']:
#     print(genre)

# # search for a person name
# people = ia.search_person('Mel Gibson')
# for person in people:
#    print(person.personID, person['name'])

## 정확한 매칭

절차
- `search_movie()`: u.item의 타이틀을 입력으로하여 검색
- 다수의 영화 검색 결과 목록중 년도가 동일하고, `plot` 항목이 있는 영화를 선택
- `plot`을 취합

결과
- movie_plots.csv
    - 123개를 매칭하지 못함
    - 검색 결과가 없거나
    - 동일 년도에 `plot`을 보유한 항목이 없거나

In [8]:
year_match = re.compile(r'.*[(](\d+)[)]$')

contents = []
for idx, x in tqdm.tqdm(item_info_df['title'].items()):
  movies=ia.search_movie(x)
  m = year_match.match(x)
  content = None
  if m:
    year = m.group(1)
    for small_movie in movies:
      if 'year' in small_movie.keys() and small_movie['year'] == int(year):
        movie=ia.get_movie(small_movie.movieID)
        if 'plot' in movie.keys():
          merged_plot = '\n'.join(movie['plot'])
          content = {'movieId':idx, 'query_title':x, 'title':movie['title'], 'plot': merged_plot}
          break
  if content is None:
    content = {'movieId':idx, 'query_title':x, 'title':x, 'plot': x}
    print(f"Errror: {idx} / {x}")

  contents.append(content)

11it [00:43,  3.11s/it]

Errror: 11 / Seven (Se7en) (1995)


16it [00:58,  2.63s/it]

Errror: 16 / French Twist (Gazon maudit) (1995)


44it [02:55,  3.24s/it]

Errror: 44 / Dolores Claiborne (1994)


94it [06:20,  4.05s/it]


KeyboardInterrupt: 

In [ ]:
pd.DataFrame.from_dict(contents).to_csv('movie_plots.csv', index=False)

## 데이터셋과 imdb에 등재된 년도가 다른 경우 보강

위에서 매칭하지 못한 123개의 영화에 대해 보강 작업
- 년도 차이를 1년까지 허용
- contents_modify_2nd.csv
    - 43개를 매칭
    - 80개는 여전히 매칭 못함


In [ ]:
merged_df = pd.read_csv('movie_plots.csv', index_col=False)

In [ ]:
year_match = re.compile(r'.*[(](\d+)[)]$')

contents = []
for idx, x in tqdm.tqdm(merged_df.query("title==plot")['title'].items()):
  movies=ia.search_movie(x)
  m = year_match.match(x)
  content = None
  if m:
    year = m.group(1)
    for small_movie in movies:
      if 'year' in small_movie.keys() and np.abs(small_movie['year']-int(year)) < 2.0:
        movie=ia.get_movie(small_movie.movieID)
        if 'plot' in movie.keys():
          merged_plot = '\n'.join(movie['plot'])
          content = {'movieId':idx, 'query_title':x, 'title':movie['title'], 'plot': merged_plot}
          break
  if content is None:
    content = {'movieId':idx, 'query_title':x, 'title':x, 'plot': x}
    print(f"Errror: {idx} / {x}")

  contents.append(content)

In [ ]:
pd.DataFrame.from_dict(contents).to_csv('contents_modify_2nd.csv', index=False)

## 합치기

최종
- movie_plots_80_missings.csv
    - 80개는 매칭하지 못하니, 타이틀을 그대로 plot화 함


In [ ]:
second_df = pd.read_csv('contents_modify_2nd.csv', index_col=False)

In [ ]:
merged_df = pd.read_csv('movie_plots.csv', index_col=False)

In [ ]:
df = pd.merge(merged_df, second_df, how='left', on=['movieId', 'query_title'])
plot = df.plot_x.copy()
plot[df.plot_y.notna()] = df.plot_y[df.plot_y.notna()]
df['plot'] = plot
df = df.drop(columns=['plot_x', 'title_y', 'plot_y'])
df.columns=['moiveId', 'query_title', 'title', 'plot']

In [ ]:
df.to_csv('movie_plots_80_missings.csv', index=False)